In [6]:
import pandas as pd
from typing import Dict, List, Tuple

class OSLogAnalyzer:
    """AOS와 iOS 로그 설계 차이점 분석기"""
    
    def __init__(self):
        self.results = []
    
    def analyze_os_differences(self, tsv_path: str, output_path: str):
        """TSV 파일에서 AOS와 iOS 로그 차이점 분석"""
        df = pd.read_csv(tsv_path, sep='\t', encoding='utf-8')
        
        for idx, row in df.iterrows():
            row_num = idx + 1
            
            # AOS와 iOS 키-값 매핑 생성
            aos_mapping = self._parse_key_value_pairs(
                row.get('keys_combined-AOS', ''), 
                row.get('values_combined-AOS', '')
            )
            
            ios_mapping = self._parse_key_value_pairs(
                row.get('keys_combined-iOS', ''), 
                row.get('values_combined-iOS', '')
            )
            
            # 기본 정보 추출
            base_info = {
                '연번': row_num,
                '기능': str(row.get('기능', '')),
                '경로': str(row.get('경로', '')),
                '활동': str(row.get('활동', '')),
                'act_type': str(row.get('act_type', '')),
                'click_type': str(row.get('click_type', ''))
            }
            
            # 차이점 분석
            self._compare_mappings(base_info, aos_mapping, ios_mapping)
        
        # 결과 저장
        self._save_results(output_path)
    
    def _parse_key_value_pairs(self, keys_str: str, values_str: str) -> Dict[str, str]:
        """키와 값 문자열을 파싱하여 매핑 생성"""
        if not keys_str or not values_str or keys_str == 'nan' or values_str == 'nan':
            return {}
        
        keys = [k.strip() for k in str(keys_str).split(',') if k.strip()]
        values = [v.strip() for v in str(values_str).split(',') if v.strip()]
        
        # 키와 값의 개수를 맞춤
        min_len = min(len(keys), len(values))
        mapping = {}
        
        for i in range(min_len):
            if keys[i] and values[i]:
                mapping[keys[i]] = values[i]
        
        return mapping
    
    def _compare_mappings(self, base_info: Dict, aos_mapping: Dict[str, str], ios_mapping: Dict[str, str]):
        """AOS와 iOS 매핑을 키 이름 기준으로 비교하여 차이점 분석"""
        all_keys = set(aos_mapping.keys()) | set(ios_mapping.keys())
        
        if not all_keys:
            # 둘 다 비어있는 경우
            self.results.append({
                **base_info,
                'key_name': 'NO_DATA',
                'aos_key': '',
                'ios_key': '',
                'aos_value': '',
                'ios_value': '',
                'key_status': 'BOTH_EMPTY',
                'value_status': 'BOTH_EMPTY',
                'overall_status': 'BOTH_EMPTY'
            })
            return
        
        # 키 이름별로 비교
        for key_name in sorted(all_keys):
            aos_value = aos_mapping.get(key_name, '')
            ios_value = ios_mapping.get(key_name, '')
            
            # 키 존재 여부에 따라 key 표시
            aos_key = key_name if key_name in aos_mapping else ''
            ios_key = key_name if key_name in ios_mapping else ''
            
            # 키와 값 각각의 상태 결정
            key_status = self._compare_key_existence(key_name, aos_mapping, ios_mapping)
            value_status = self._compare_values(aos_value, ios_value, key_name, aos_mapping, ios_mapping)
            overall_status = self._determine_overall_status(key_status, value_status)
            
            self.results.append({
                **base_info,
                'key_name': key_name,
                'aos_key': aos_key,
                'ios_key': ios_key,
                'aos_value': aos_value,
                'ios_value': ios_value,
                'key_status': key_status,
                'value_status': value_status,
                'overall_status': overall_status
            })
    
    def _compare_key_existence(self, key_name: str, aos_mapping: Dict, ios_mapping: Dict) -> str:
        """키 존재 여부 비교"""
        aos_exists = key_name in aos_mapping
        ios_exists = key_name in ios_mapping
        
        if aos_exists and ios_exists:
            return 'PASS'
        elif aos_exists and not ios_exists:
            return 'AOS_ONLY'
        elif not aos_exists and ios_exists:
            return 'IOS_ONLY'
        else:
            return 'BOTH_MISSING'  # 이론상 발생하지 않음
    
    def _compare_values(self, aos_value: str, ios_value: str, key_name: str, aos_mapping: Dict, ios_mapping: Dict) -> str:
        """값 비교 (키가 양쪽에 모두 있을 때만 비교)"""
        aos_exists = key_name in aos_mapping
        ios_exists = key_name in ios_mapping
        
        if not aos_exists and not ios_exists:
            return 'BOTH_MISSING'
        elif aos_exists and not ios_exists:
            return 'AOS_ONLY'
        elif not aos_exists and ios_exists:
            return 'IOS_ONLY'
        else:
            # 둘 다 키가 존재하는 경우 값 비교
            if aos_value == ios_value:
                return 'PASS'
            else:
                return 'FAIL'
    
    def _determine_overall_status(self, key_status: str, value_status: str) -> str:
        """전체 상태 결정"""
        if key_status == 'BOTH_EMPTY' and value_status == 'BOTH_EMPTY':
            return 'BOTH_EMPTY'
        elif key_status == 'PASS' and value_status == 'PASS':
            return 'PASS'
        else:
            return 'FAIL'
    
    def _save_results(self, output_path: str):
        """분석 결과를 Excel 파일로 저장"""
        df = pd.DataFrame(self.results)
        
        # 컬럼 순서 정의
        column_order = [
            '연번', '기능', '경로', '활동', 'act_type', 'click_type',
            'key_name', 'aos_key', 'ios_key', 'key_status',
            'aos_value', 'ios_value', 'value_status', 'overall_status'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in column_order if col in df.columns]
        df = df[available_columns]
        
        df.to_excel(output_path, index=False, engine='openpyxl')
        
        # 통계 요약
        total = len(df)
        passed = len(df[df['overall_status'] == 'PASS'])
        failed = len(df[df['overall_status'] == 'FAIL'])
        
        return {
            'total': total,
            'passed': passed,
            'failed': failed,
            'pass_rate': passed / total * 100 if total > 0 else 0
        }
    
    def generate_summary_report(self, output_path: str):
        """차이점 요약 보고서 생성"""
        df = pd.DataFrame(self.results)
        
        # 전체 상태별 통계
        overall_stats = df['overall_status'].value_counts().to_dict()
        key_stats = df['key_status'].value_counts().to_dict()
        value_stats = df['value_status'].value_counts().to_dict()
        
        # 연번별 차이점 개수
        diff_count_by_row = df[df['overall_status'] == 'FAIL'].groupby('연번').size().reset_index()
        diff_count_by_row.columns = ['연번', '차이점_개수']
        
        # 기능별 차이점 통계
        diff_by_function = df[df['overall_status'] == 'FAIL'].groupby('기능').size().reset_index()
        diff_by_function.columns = ['기능', '차이점_개수']
        
        # 키별 차이점 통계
        key_diff_stats = df[df['key_status'].isin(['AOS_ONLY', 'IOS_ONLY'])]['key_name'].value_counts().reset_index()
        key_diff_stats.columns = ['키이름', '차이점_개수']
        
        # Excel 파일로 저장 (여러 시트)
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # 전체 결과
            pd.DataFrame(self.results).to_excel(writer, sheet_name='전체결과', index=False)
            
            # 실패 케이스만
            fail_df = df[df['overall_status'] == 'FAIL']
            fail_df.to_excel(writer, sheet_name='차이점', index=False)
            
            # AOS에만 있는 키들
            aos_only_df = df[df['key_status'] == 'AOS_ONLY']
            aos_only_df.to_excel(writer, sheet_name='AOS전용키', index=False)
            
            # iOS에만 있는 키들
            ios_only_df = df[df['key_status'] == 'IOS_ONLY']
            ios_only_df.to_excel(writer, sheet_name='iOS전용키', index=False)
            
            # 값이 다른 경우들
            value_diff_df = df[df['value_status'] == 'FAIL']
            value_diff_df.to_excel(writer, sheet_name='값차이점', index=False)
            
            # 통계 요약
            summary_data = [
                {'구분': '전체상태_PASS', '개수': overall_stats.get('PASS', 0)},
                {'구분': '전체상태_FAIL', '개수': overall_stats.get('FAIL', 0)},
                {'구분': '키상태_PASS', '개수': key_stats.get('PASS', 0)},
                {'구분': '키상태_AOS_ONLY', '개수': key_stats.get('AOS_ONLY', 0)},
                {'구분': '키상태_IOS_ONLY', '개수': key_stats.get('IOS_ONLY', 0)},
                {'구분': '값상태_PASS', '개수': value_stats.get('PASS', 0)},
                {'구분': '값상태_FAIL', '개수': value_stats.get('FAIL', 0)}
            ]
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='통계요약', index=False)
            
            # 연번별 차이점
            diff_count_by_row.to_excel(writer, sheet_name='연번별차이점', index=False)
            
            # 기능별 차이점
            diff_by_function.to_excel(writer, sheet_name='기능별차이점', index=False)
            
            # 키별 차이점
            key_diff_stats.to_excel(writer, sheet_name='키별차이점', index=False)


def analyze_os_log_differences(input_file: str, output_file: str, summary_file: str):
    """OS별 로그 차이점 분석 실행"""
    analyzer = OSLogAnalyzer()
    
    # 차이점 분석
    stats = analyzer.analyze_os_differences(input_file, output_file)
    
    # 요약 보고서 생성
    analyzer.generate_summary_report(summary_file)
    
    return stats

# 실행 예시
if __name__ == "__main__":
    stats = analyze_os_log_differences(
        "OSdiff/paste.txt",  # 입력 TSV 파일
        "./result/os_comparison_result.xlsx",  # 기본 결과
        "./result/os_comparison_summary.xlsx"  # 요약 보고서
    )